# Unit Testing of File Reader and Language Processing

This notebook contains unit tests for functions contained within the fileReader.py and languageProcessing.py files. The GUI can be tested by directly running the gui.py file.

## Imports and Setup

In [49]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('maxent_ne_chunker')
nltk.download('words')
%pip install pypdf2

from languageProcessing import *
from fileReader import *



[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/nicholasderby/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/nicholasderby/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nicholasderby/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nicholasderby/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/nicholasderby/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/nicholasderby/nltk_data...
[nltk_data]   Package words is al

Note: you may need to restart the kernel to use updated packages.


## File Reader Tests

### Testing to ensure PDF reader functions properly

Though fileReader.py does contain additional functions, they must be called directly by the GUI.

In [50]:
inputPDF = "test_files/colorado-wikipedia-intro.pdf"
outputTXT = "test_files/colorado-wikipedia-intro.txt"
controlTXT = "test_files/colorado-intro-wiki.txt"

pdf_to_txt(inputPDF, outputTXT)

with open(outputTXT, 'r') as file:
    outputText = file.read()

with open(controlTXT, 'r') as file:
    controlText = file.read()

for outputWord, controlWord in zip(outputText.split(), controlText.split()):
    assert(outputWord == controlWord)

## Language Processing Tests

Test sentence detection. Since every sentence in the sample text ends in a period, checking that each detected sentence ends in a period is an effective test in this case.

In [51]:
text = controlText  # renaming for readability

sentences = sentence_detection(text)

for sentence in sentences:
    print(sentence)
    assert(sentence[len(sentence) - 1] == '.') 


Colorado is a state in the Western United States.
It is one of the Mountain states, sharing the Four Corners region with Arizona, New Mexico, and Utah.
It is also bordered by Wyoming to the north, Nebraska to the northeast, Kansas to the east, and Oklahoma to the southeast.
Colorado is noted for its landscape of mountains, forests, high plains, mesas, canyons, plateaus, rivers, and desert lands.
It encompasses most of the Southern Rocky Mountains, as well as the northeastern portion of the Colorado Plateau and the western edge of the Great Plains.
Colorado is the eighth most extensive and 21st most populous U.S. state.
The United States Census Bureau estimated the population of Colorado at 5,877,610 as of July 1, 2023, a 1.80% increase since the 2020 United States census.
The region has been inhabited by Native Americans and their ancestors for at least 13,500 years and possibly much longer.
The eastern edge of the Rocky Mountains was a major migration route for early peoples who sprea

Test tokenize, checking that each token properly matches up with the original text.

In [52]:
words = text.split()
tokens = tokenize(text)

for token in tokens:
    if token == "``" or token == "''":      # ignore quotes since tokenizer reformats them
        continue

    match = False
    for word in words:
        if token in word:
            match = True

    assert(match)

Test part_of_speech_tagging, ensuring that its output is as expected.

In [53]:
tags = part_of_speech_tagging(text)

for tag in tags:
    print(tag, end="")

Colorado [NNP] is [VBZ] a [DT] state [NN] in [IN] the [DT] Western [NNP] United [NNP] States [NNPS] . [.] It [PRP] is [VBZ] one [CD] of [IN] the [DT] Mountain [NNP] states [NNS] , [,] sharing [VBG] the [DT] Four [CD] Corners [NNPS] region [NN] with [IN] Arizona [NNP] , [,] New [NNP] Mexico [NNP] , [,] and [CC] Utah [NNP] . [.] It [PRP] is [VBZ] also [RB] bordered [VBN] by [IN] Wyoming [VBG] to [TO] the [DT] north [NN] , [,] Nebraska [NNP] to [TO] the [DT] northeast [NN] , [,] Kansas [NNP] to [TO] the [DT] east [NN] , [,] and [CC] Oklahoma [NNP] to [TO] the [DT] southeast [NN] . [.] Colorado [NNP] is [VBZ] noted [VBN] for [IN] its [PRP$] landscape [NN] of [IN] mountains [NNS] , [,] forests [NNS] , [,] high [JJ] plains [NNS] , [,] mesas [NN] , [,] canyons [NNS] , [,] plateaus [NN] , [,] rivers [NNS] , [,] and [CC] desert [NN] lands [NNS] . [.] It [PRP] encompasses [VBZ] most [JJS] of [IN] the [DT] Southern [NNP] Rocky [NNP] Mountains [NNP] , [,] as [RB] well [RB] as [IN] the [DT] northea

Test word_frequency function, comparing its results to a simple command F search on the original file.

In [54]:
frequencies = word_frequency(text)
assert("colorado {11}" in frequencies)

Test convert_pos_to_wordnet to ensure that it accurately converts nltk part of speech tags to wordnet part of speech tags.

In [55]:
assert(convert_pos_to_wordnet('JS') == 'a')
assert(convert_pos_to_wordnet('VB') == 'v')
assert(convert_pos_to_wordnet('NP') == 'n')
assert(convert_pos_to_wordnet('RS') == 'r')
assert(convert_pos_to_wordnet('XL') == 'n')

Test lemmatization, ensuring a conjugated verb such as 'is' is properly lemmatized to 'be'.

In [56]:
lemmas = lemmatization(text)

for lemma, token in zip(lemmas, token):
    if token == 'is':
        assert lemma == 'be'

Test get_first_sentence.

In [57]:
assert(get_first_sentence() == "Colorado is a state in the Western United States.")

In [58]:
entities = named_entity_recognition(text)
assert("GPE Colorado" in entities)